In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

/kaggle/input/the-boston-houseprice-data/boston.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Предварительная обработка данных

1. Посмотреть на пропуски. 

In [2]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
missing_columns = df.columns[df.isnull().any()]
print(missing_columns)

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')


In [4]:
print(df.info())  # Сведения о типах данных и пропусках
print(df.describe())  # Статистическое описание числовых колонок

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Посмотрим на уникальные значения признаков с типом object
> *Let's look at the unique values of features with the object data type.*

In [5]:
object_columns = df.select_dtypes(include=['object'])

for column in object_columns.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print('-' * 50)  # Separator for readability


Unique values in column 'MSZoning':
['RL' 'RM' 'C (all)' 'FV' 'RH']
--------------------------------------------------
Unique values in column 'Street':
['Pave' 'Grvl']
--------------------------------------------------
Unique values in column 'Alley':
[nan 'Grvl' 'Pave']
--------------------------------------------------
Unique values in column 'LotShape':
['Reg' 'IR1' 'IR2' 'IR3']
--------------------------------------------------
Unique values in column 'LandContour':
['Lvl' 'Bnk' 'Low' 'HLS']
--------------------------------------------------
Unique values in column 'Utilities':
['AllPub' 'NoSeWa']
--------------------------------------------------
Unique values in column 'LotConfig':
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
--------------------------------------------------
Unique values in column 'LandSlope':
['Gtl' 'Mod' 'Sev']
--------------------------------------------------
Unique values in column 'Neighborhood':
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 

## Обработаем числовые признаки

In [6]:
# Фасад участка: Линейный отрезок улицы, соединенный с объектом недвижимости
df['LotFrontage'].fillna(df['LotFrontage'].median(), inplace = True)

/tmp/ipykernel_17/2685540953.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LotFrontage'].fillna(df['LotFrontage'].median(), inplace = True)


## Аккуратно обработаем объектные признаки

In [7]:
# Выход к аллее
df['Alley'].fillna('None', inplace = True)

/tmp/ipykernel_17/2758298995.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Alley'].fillna('None', inplace = True)


**Обрати внимание**

25  MasVnrType     588 non-null    object 

26  MasVnrArea     1452 non-null   float64

In [8]:
# Тип кладки: Тип облицовки каменной кладки
df[df['MasVnrType'].isna()][['Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea']]

,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea
1,MetalSd,MetalSd,NaN,0.0
3,Wd Sdng,Wd Shng,NaN,0.0
5,VinylSd,VinylSd,NaN,0.0
8,BrkFace,Wd Shng,NaN,0.0
9,MetalSd,MetalSd,NaN,0.0
...,...,...,...,...
1454,VinylSd,VinylSd,NaN,0.0
1455,VinylSd,VinylSd,NaN,0.0
1457,CemntBd,CmentBd,NaN,0.0
1458,MetalSd,MetalSd,NaN,0.0


**Для этих значение заполним модой**

In [9]:
df[df['MasVnrType'].isna()][df['MasVnrArea'] > 0][['MasVnrType', 'MasVnrArea']]

/tmp/ipykernel_17/1617102378.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['MasVnrType'].isna()][df['MasVnrArea'] > 0][['MasVnrType', 'MasVnrArea']]


,MasVnrType,MasVnrArea
624,NaN,288.0
773,NaN,1.0
1230,NaN,1.0
1300,NaN,344.0
1334,NaN,312.0


In [10]:
# Найти наиболее частое значение
most_frequent_value = df['MasVnrType'].mode()
most_frequent_value

0    BrkFace
Name: MasVnrType, dtype: object

**Для остальных None**

In [11]:
# Заполнить пропуски специальной категорией 'None'
df['MasVnrType'].fillna('None', inplace=True)

/tmp/ipykernel_17/421740333.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['MasVnrType'].fillna('None', inplace=True)


In [12]:
df['MSZoning'].unique()

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [13]:
df['PoolQC'].unique()
# titanic.Survived.unique()

array([nan, 'Ex', 'Fa', 'Gd'], dtype=object)